# EG 24/25 - Conversão de Expressão para Lark

## Maria Cunha e Tomás Campinho - Gr10

## Exemplo fornecido pelos Docentes

In [4]:
!pip install lark
from lark import Lark
from lark.tree import pydot__tree_to_png

grammar2 = '''
// Regras Sintaticas
start: PE ( | elementos) PD
elementos : elemento (VIR elemento)*
elemento : NUMERO

// Regras Lexicográficas
NUMERO:"0".."9"+ // [0-9]+
PE:"["
PD:"]"
VIR:","

// Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''

frase = "[1,23,345]"

p = Lark(grammar2) # cria um objeto parser

tree = p.parse(frase)  # retorna uma tree
print(tree)
print(tree.pretty())
#pydot__tree_to_png(tree,'lark_test.png')

Tree(Token('RULE', 'start'), [Token('PE', '['), Tree(Token('RULE', 'elementos'), [Tree(Token('RULE', 'elemento'), [Token('NUMERO', '1')]), Token('VIR', ','), Tree(Token('RULE', 'elemento'), [Token('NUMERO', '23')]), Token('VIR', ','), Tree(Token('RULE', 'elemento'), [Token('NUMERO', '345')])]), Token('PD', ']')])
start
  [
  elementos
    elemento	1
    ,
    elemento	23
    ,
    elemento	345
  ]



## Conversão da expressão [1.0:9.0][15.5:19.0] para Lark

In [ ]:
from lark import Lark

grammar = '''
// Regras Sintáticas
start: intervalo+        // Um ou mais intervalos
intervalo: PE NUMERO DP NUMERO PD   // Define um intervalo [num:num]

// Regras Lexicográficas
NUMERO: /\d+(\.\d+)?/   
PE: "["
PD: "]"
DP: ":"

// Ignorar espaços em branco
%import common.WS
%ignore WS
'''

# Expressão de exemplo, obtida da aula dos docentes
frase = "[1.0:9.0][15.5:19.0]"


parser = Lark(grammar, start='start')


tree = parser.parse(frase)
print(tree.pretty())  


start
  intervalo
    [
    1.0
    9.0
    ]
  intervalo
    [
    15.5
    19.0
    ]

